In [18]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
try:
    import empyrical as emp
except:
    emp = None
import tushare as ts
import time
import random
# from concurrent.futures import ProcessPoolExecutor

from common.log import *
from common.config import Config
from spider.spider_nasdaq import Spider_nasdaq
from spider.spider_coinmarketcap import Spider_coinmarketcap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from pandas_highcharts.core import serialize
from pandas_highcharts.display import display_charts

CONF = Config('../conf/secret.yaml').data[0]
ts_token = CONF['TUSHARE']['TOKEN']
ts.set_token(ts_token)
pro = ts.pro_api()

CONF = Config().data[0]
MONGODB = CONF['MONGODB']
NASDAQ = CONF['NASDAQ']
CRYPTOCURRENCY = CONF['CRYPTOCURRENCY']
CRYPTOCURRENCY = list(CRYPTOCURRENCY.keys())
NASDAQ100 = CONF['NASDAQ100']
HS300 = list(ts.get_hs300s()['code'])
HS300_df = pd.read_csv('../database/HS300IDX_ALL.csv')
HS300 = list(set(HS300_df.con_code))
HS300 = [x.split('.')[0] for x in HS300]
ZZ500 = list(ts.get_zz500s()['code'])
SZ50 = list(ts.get_sz50s()['code'])
HSALL = list(set(ts.get_stock_basics().index) - set(['601162', '002940', '002939', '300674', '002941']))
HSALL = HSALL + ['000866', '600001', '000956']

BENCHMARK = '399300'
# TARGET = ['399300']
TARGET = HS300
TARGET = ['000814']
# BENCHMARK = '000905'
# TARGET = ZZ500
# BENCHMARK = '000016'
# TARGET = SZ50
# TARGET = HSALL

# BENCHMARK = 'NDX'
# TARGET = NASDAQ100

# BENCHMARK = 'BITCOIN'
# TARGET = CRYPTOCURRENCY
# BENCHMARK = 'EOS'
# TARGET = ['EOS']

ALL_TARGET = TARGET[:]

pd
len(TARGET)
TARGET[:5]

d:\python36\lib\site-packages\tushare\stock\classifying.py:266: FutureWarning: the 'parse_cols' keyword is deprecated, use 'usecols' instead
  ct.PAGES['sz50b']), parse_cols=[0, 4, 5])


<module 'pandas' from 'd:\\python36\\lib\\site-packages\\pandas\\__init__.py'>

1

['000814']

In [ ]:
SZ50_20121212 = ['600000', '600010', '600015', '600016', '600019', '600028',
 '600030', '600031', '600036', '600048', '600050', '600058',
 '600068', '600089', '600104', '600111', '600188', '600348',
 '600362', '600383', '600489', '600519', '600547', '600585',
 '600837', '600900', '601006', '601088', '601111', '601118',
 '601166', '601168', '601169', '601288', '601299', '601318',
 '601328', '601398', '601600', '601601', '601628', '601668',
 '601699', '601766', '601818', '601857', '601898', '601899',
 '601958', '601989']

SZ50_20151204 = ['600000', '600010', '600015', '600016', '600018', '600028',
'600030', '600036', '600048', '600050', '600111', '600104',
'600109', '600111', '600150', '600256', '600406', '600518',
'600519', '600583', '600585', '600637', '600690', '600837',
'600887', '600893', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601288', '601318', '601328',
'601390', '601398', '601601', '601628', '601668', '601688',
'601766', '601800', '601818', '601857', '601901', '601988',
'601989', '601998']

SZ50_20160325 = ['600000', '600010', '600015', '600016', '600018', '600028',
'600030', '600036', '600048', '600050', '600104', '600109',
'600111', '600150', '600518', '600519', '600585', '600637',
'600795', '600837', '600887', '600893', '600958', '600999',
'601006', '601088', '601166', '601169', '601186', '601211',
'601288', '601318', '601328', '601336', '601390', '601398',
'601601', '601628', '601668', '601669', '601688', '601766',
'601800', '601818', '601857', '601901', '601985', '601988',
'601989', '601998']

SZ50_20161212 = ['600000', '600016', '600028', '600029', '600030', '600036',
'600048', '600050', '600100', '600104', '600109', '600111',
'600485', '600518', '600519', '600547', '600637', '600837',
'600887', '600893', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601198', '601211', '601288',
'601318', '601328', '601336', '601377', '601390', '601398',
'601601', '601628', '601668', '601688', '601766', '601788',
'601800', '601818', '601857', '601901', '601985', '601988',
'601989', '601998']

SZ50_20171213 = ['600000', '600016', '600019', '600028', '600029', '600030',
'600036', '600048', '600050', '600104', '600111', '600309',
'600340', '600518', '600519', '600547', '600606', '600837',
'600887', '600919', '600958', '600999', '601006', '601088',
'601166', '601169', '601186', '601211', '601229', '601288',
'601318', '601328', '601336', '601390', '601398', '601601',
'601628', '601668', '601669', '601688', '601766', '601800',
'601818', '601857', '601878', '601881', '601985', '601988',
'601989', '603993']

FULL_SZ50 = SZ50_20121212 + SZ50_20151204 + SZ50_20160325 + SZ50_20161212 + SZ50_20171213 + list(ts.get_sz50s()['code'])
FULL_SZ50 = list(set(FULL_SZ50))

# TARGET = FULL_SZ50
# ALL_TARGET = TARGET[:]

In [13]:
df = pro.fina_indicator(ts_code='002151.SZ', fields='ts_code,ann_date,end_date,eps,dt_eps,bps,roe,roe_waa,roe_dt,roa,q_eps,q_roe,q_dt_roe')
df.columns
df

Index(['ts_code', 'ann_date', 'end_date', 'eps', 'dt_eps', 'bps', 'roe',
       'roe_waa', 'roe_dt', 'roa', 'q_eps', 'q_roe', 'q_dt_roe'],
      dtype='object')

,ts_code,ann_date,end_date,eps,dt_eps,bps,roe,roe_waa,roe_dt,roa,q_eps,q_roe,q_dt_roe
0,002151.SZ,20181023,20180930,0.1000,0.1000,8.5521,1.2286,1.17,-0.4193,1.7304,0.0108,0.1269,-0.5144
1,002151.SZ,20180829,20180630,0.0900,0.0900,8.4496,1.1083,1.10,0.0956,1.3003,0.0858,1.0168,0.5007
2,002151.SZ,20180426,20180331,0.0079,0.0079,8.4306,0.0930,0.09,-0.4049,0.2253,0.0079,0.0930,-0.4049
3,002151.SZ,20180419,20171231,0.2100,0.2000,8.4534,2.4264,2.43,0.8127,2.4617,0.0567,0.6729,0.4980
4,002151.SZ,20171027,20170930,0.1500,0.1500,8.4112,1.7584,1.75,0.3154,1.5729,0.0981,1.1725,0.2361
5,002151.SZ,20170824,20170630,0.0500,0.0500,8.3162,0.5953,0.60,0.0810,0.4510,0.0427,0.5103,0.2386
6,002151.SZ,20170429,20170331,0.0100,0.0100,8.4000,0.0847,0.08,-0.1569,0.1002,0.0071,0.0847,-0.1569
7,002151.SZ,20170428,20161231,0.1100,0.1100,8.4148,1.4867,1.49,1.5675,1.8425,-0.0008,-0.0090,0.4365
8,002151.SZ,20161026,20160930,0.1100,0.1100,8.4946,1.4994,1.63,1.0281,1.4650,0.0771,0.9118,0.6812
9,002151.SZ,20160824,20160630,0.0300,0.0300,8.4165,0.3774,0.49,0.1893,0.4943,0.0480,0.7050,0.6108


In [19]:
%%time

IS_DOWNLOAD_DATA = True

for symbol in TARGET + [BENCHMARK]:
    if not IS_DOWNLOAD_DATA:
        break
        
    print(symbol)

    # 指数
    if symbol in ['399300', '000905', '000016']:
        df = ts.get_k_data(code=symbol, index=True, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # A股
    elif symbol in HS300 or symbol in ZZ500 or symbol in HSALL:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        time.sleep(1)

        if symbol.startswith('00') or symbol.startswith('30'):
            temp_symbol = '%s.SZ' % symbol
        elif symbol.startswith('60'):
            temp_symbol = '%s.SH' % symbol
            
        df = pro.daily_basic(ts_code=temp_symbol, start_date='20050101', end=time.strftime('%Y%m%d'))
        datafile = '../database/market/%s_pro_daily_basic.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
        fields='ts_code,ann_date,end_date,eps,dt_eps,bps,roe,roe_waa,roe_dt,roa,q_eps,q_roe,q_dt_roe'
        df = pro.fina_indicator(ts_code=temp_symbol, fields=fields)
        datafile = '../database/market/%s_pro_fina_indicator.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        
    # ETF
    else:
        df = ts.get_k_data(code=symbol, ktype='D', autype='qfq', index=False, start='2005-01-01', end=time.strftime('%Y-%m-%d'))
        datafile = '../database/market/%s.csv' % symbol
        df.to_csv(datafile, index=False, encoding='utf-8')
        time.sleep(1)
        
    time.sleep(1)

000814
399300
Wall time: 4.19 s


In [ ]:
full_df.to_csv('../database/HS300IDX_2007-2009.csv', index=False, encoding='utf-8')

In [ ]:
pro.index_weight(index_code='000905.SH', start_date='2005-01-01', end_date='2007-01-01')

In [ ]:
flag = 0
full_df = None
for today in pd.period_range(start='2015-01-01', end='2019-01-01', freq='D'):
    if flag == 0:
        start_date = today.strftime('%Y%m%d')
        flag = 1
    elif flag == 1:
        end_date = today.strftime('%Y%m%d')
        df = pro.index_weight(index_code='000905.SH', start_date=start_date, end_date=end_date)
        if full_df is not None:
            full_df = pd.concat([full_df, df], ignore_index=True)
        else:
            full_df = df.copy()
        if len(df) != 0:
            today , len(df), len(full_df)
            pass
        flag = 0
        time.sleep(1)

len(full_df)
full_df.to_csv('../database/ZZ500IDX_2015-2018.csv', index=False, encoding='utf-8')
full_df



# start_date = '20050101'
# end_date = '20050201'
# pro.index_weight(index_code='399300.SZ', start_date=start_date, end_date=end_date)